In [1]:
from pyscf import gto,scf
import numpy as np
import matplotlib.pyplot as plt
from FcMole import *

In [2]:
CO=gto.M(atom="C 0 0 0; O 0 0 2 ",unit="Bohr", basis="sto-3g")

In [3]:
%%time
mf=scf.RKS(CO)
mf.xc="PBE0"
mf.scf()

converged SCF energy = -111.621701280108
CPU times: user 6.22 s, sys: 235 ms, total: 6.46 s
Wall time: 193 ms


-111.62170128010848

In [4]:
mf.e_tot

-111.62170128010848

In [5]:
summary=dict(mf.scf_summary)
summary

{'e1': -199.4162294162503,
 'coul': 77.70084988123735,
 'exc': -13.906321745095527,
 'nuc': 24.0}

In [6]:
sum(summary.values()) ,mf.e_tot

(-111.62170128010848, -111.62170128010848)

In [7]:
ni=mf._numint
omega, alpha, hyb = ni.rsh_and_hybrid_coeff(mf.xc, spin=mf.mol.spin)
omega, alpha, hyb

(0.0, 0.25, 0.25)

In [8]:
#P=CC^T
C=mf.mo_coeff
Pc=2*C[:,:7].dot(C.T[:7,:])
P0=mf.make_rdm1()

In [9]:
J,K=mf.get_jk()
h1=mf.get_hcore()
F=mf.get_fock()

In [10]:
summary["e1"],np.einsum("ij,ij",h1,P0)

(-199.4162294162503, -199.41622941625033)

In [11]:
summary["coul"],np.einsum("ij,ij",J,P0)/2 # it is the HF Coulomb operator 

(77.70084988123735, 77.70084988123736)

In [12]:
summary["exc"],-np.einsum("ij,ij",K,P0)/4    # DFT XC energy / HF exchange

(-13.906321745095527, -13.481304665613102)

In [13]:
V_eff=mf.get_veff()
-np.einsum("ij,ij",V_eff.vk,P0),np.allclose(V_eff.vk,K/4)

(-13.481304665613102, True)

In [14]:
summary["exc"],V_eff.exc

(-13.906321745095527, -13.906321745095527)

In [15]:
ni=mf._numint

In [16]:
ks=mf
mol=CO
dm=P0
n, exc, vxc = ni.nr_rks(mol, ks.grids, ks.xc, dm)
n, exc#, vxc 

(14.00000014868435, -10.535995578692251)

In [17]:
np.einsum("ij,ji",P0,mol.get_ovlp())

converged SCF energy = -111.199724042754


13.999999999999995

In [18]:
np.einsum('ij,ji', dm, vxc).real #  the potential does not retrieve the energy after DM contraction

-13.547910378628355

In [19]:
exc,exc- np.einsum('ij,ji', dm, V_eff.vk).real * .5 * .5,summary["exc"],

(-10.535995578692251, -13.906321745095527, -13.906321745095527)

In [20]:
omega, alpha, hyb = ni.rsh_and_hybrid_coeff(ks.xc, spin=mol.spin)
omega, alpha,hyb

(0.0, 0.25, 0.25)

In [21]:
xctype=ni._xc_type(mf.xc) # GGA
ks.direct_scf,ni.libxc.is_hybrid_xc(ks.xc),ni._xc_type(mf.xc)

(True, True, 'GGA')

In [22]:
grid=mf.grids
vars(mf.grids)

{'mol': <pyscf.gto.mole.Mole at 0x7f6ac50ee850>,
 'stdout': <ipykernel.iostream.OutStream at 0x7f6afecb1610>,
 'verbose': 3,
 'symmetry': False,
 'coords': array([[-4.46851140e+00, -4.46851140e+00, -2.37126186e+00],
        [-4.78551877e+00, -4.78551877e+00, -2.53948510e+00],
        [-4.48132694e+00, -4.48132694e+00, -2.48132694e+00],
        ...,
        [ 1.00000000e+04,  1.00000000e+04,  1.00000000e+04],
        [ 1.00000000e+04,  1.00000000e+04,  1.00000000e+04],
        [ 1.00000000e+04,  1.00000000e+04,  1.00000000e+04]]),
 'weights': array([1.09499036, 1.38921666, 0.01994748, ..., 0.        , 0.        ,
        0.        ]),
 'non0tab': array([[ 0, 78, 86,  0, 64, 72],
        [ 0, 83, 90,  0, 72, 80],
        [ 0, 87, 93,  0, 78, 85],
        ...,
        [ 0, 83, 90,  0, 72, 80],
        [ 0, 78, 86,  0, 66, 74],
        [ 0, 70, 78,  0, 57, 65]], dtype=uint8),
 'screen_index': array([[ 0, 78, 86,  0, 64, 72],
        [ 0, 83, 90,  0, 72, 80],
        [ 0, 87, 93,  0, 78, 85

In [23]:
from pyscf.dft.numint import eval_rho,eval_ao

In [24]:
ni.block_loop(0,ks.grids)
ni.eval_ao

<function pyscf.dft.numint.eval_ao(mol, coords, deriv=0, shls_slice=None, non0tab=None, cutoff=None, out=None, verbose=None)>

In [25]:
ni.eval_ao(CO,np.array([[0,0,0],[10,10,10]])) # eval every ao in the points

array([[ 6.14288519e+00,  2.15554817e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  8.20476263e-12,
         5.57535838e-02,  0.00000000e+00,  0.00000000e+00,
        -9.24775489e-02],
       [ 0.00000000e+00,  1.76395084e-30,  9.31205491e-30,
         9.31205491e-30,  9.31205491e-30,  0.00000000e+00,
         5.89152862e-45,  4.06856395e-44,  4.06856395e-44,
         3.25485116e-44]])

In [26]:
ni.eval_ao(CO,np.array([[0,0,0],[10,10,10]]),deriv=1).shape # for gga also the gradient has to be computed, 4 dim

(4, 2, 10)

In [27]:
ao_ong=ni.eval_ao(CO,mf.grids.coords,deriv=1)

In [28]:
ni._gen_rho_evaluator(CO,P0)

(<function pyscf.dft.numint.NumInt._gen_rho_evaluator.<locals>.make_rho(idm, ao, sindex, xctype)>,
 1,
 10)

In [29]:
make_rho, nset, nao = ni._gen_rho_evaluator(mol, dm, True, False, ks.grids)

In [30]:
make_rho

<function pyscf.dft.numint.NumInt._gen_rho_evaluator.<locals>.make_rho(idm, ao, sindex, xctype)>

In [31]:
%%time
make_rho([0], ao_ong ,None, "GGA")

CPU times: user 640 ms, sys: 0 ns, total: 640 ms
Wall time: 14.6 ms


array([[3.97719526e-10, 2.23716826e-11, 2.89496971e-10, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.53592263e-09, 9.27729828e-11, 1.12189014e-09, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.53592263e-09, 9.27729828e-11, 1.12189014e-09, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.69982024e-10, 4.68852934e-11, 5.88672486e-10, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [32]:
%%time
eval_rho(mol, ao_ong , P0, xctype="GGA")

CPU times: user 1.01 s, sys: 0 ns, total: 1.01 s
Wall time: 20.4 ms


array([[3.97719526e-10, 2.23716826e-11, 2.89496971e-10, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.53592263e-09, 9.27729828e-11, 1.12189014e-09, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.53592263e-09, 9.27729828e-11, 1.12189014e-09, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.69982024e-10, 4.68852934e-11, 5.88672486e-10, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

# Get the energies

In [33]:
rho_g=eval_rho(mol, ao_ong , P0, xctype="GGA")

In [34]:
xc_code="PBE0"
ni.eval_xc_eff(xc_code, rho_g, deriv=1, xctype=xctype)

(array([-0.00073487, -0.00028157, -0.00066104, ...,  0.        ,
         0.        ,  0.        ]),
 array([[-9.79819136e-04, -3.75423085e-04, -8.81390887e-04, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-1.70158145e-10, -7.70214460e-12, -1.19995647e-10, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-1.70158145e-10, -7.70214460e-12, -1.19995647e-10, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-8.53029376e-11, -3.89248355e-12, -6.29635055e-11, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]),
 None,
 None)

In [51]:
exc, vxc=ni.eval_xc_eff(xc_code, rho_g, deriv=1, xctype=xctype)[:2]
vxc*=ks.grids.weights

In [36]:
den=rho_g[0]*ks.grids.weights

In [37]:
den.sum()

14.00000014868435

In [42]:
np.dot(den, exc),np.dot(den, exc)-np.einsum("ij,ij",P0,K)/16  ,summary['exc']

(-10.535995578692251, -13.906321745095527, -13.906321745095527)

In [39]:
exc.shape,vxc.shape

((23136,), (4, 23136))

In [43]:
n0, exc0, vxc0 = ni.nr_rks(mol, ks.grids, ks.xc, dm)
n0

14.00000014868435

In [45]:
vxc0.shape

(10, 10)

In [72]:
vxc0

array([[-1.71996375e+00, -2.41057629e-01, -1.95156391e-18,
        -6.01732206e-18,  2.70071249e-04, -1.58760958e-05,
        -5.83843075e-02,  2.56142763e-18,  3.04931861e-18,
         9.65667079e-02],
       [-2.41057629e-01, -4.50376033e-01,  5.26244629e-17,
        -9.70360944e-17, -3.87927154e-02, -7.12940610e-02,
        -2.46664657e-01, -3.75405002e-18, -1.74556550e-17,
         1.99171887e-01],
       [-1.95156391e-18,  5.26244629e-17, -4.19073117e-01,
        -1.73472348e-18,  2.16840434e-19,  2.75073950e-18,
        -2.53432258e-18, -1.31797807e-01,  3.98783112e-18,
         1.25225351e-17],
       [-6.01732206e-18, -9.70360944e-17, -1.73472348e-18,
        -4.19073117e-01,  5.85469173e-17,  1.74488787e-18,
        -6.39679282e-18,  3.66595860e-18, -1.31797807e-01,
         2.21177243e-17],
       [ 2.70071249e-04, -3.87927154e-02,  2.16840434e-19,
         5.85469173e-17, -5.01561204e-01, -1.23010517e-01,
        -3.01330458e-01, -2.25514052e-17, -1.64798730e-17,
         1.

In [77]:
F-h1-J+K/2/4-vxc0

array([[ 2.44249065e-15,  1.38777878e-16,  0.00000000e+00,
        -1.61778115e-32, -2.66171633e-17,  1.42301535e-19,
        -4.85722573e-17,  0.00000000e+00,  7.70371978e-34,
        -1.11022302e-16],
       [ 1.38777878e-16, -1.33226763e-15,  0.00000000e+00,
         1.23259516e-31,  9.71445147e-17, -2.91433544e-16,
         1.66533454e-16,  7.70371978e-34,  2.46519033e-32,
         1.66533454e-16],
       [ 0.00000000e+00,  0.00000000e+00,  9.43689571e-16,
         1.54074396e-33, -7.70371978e-34,  7.70371978e-34,
         3.85185989e-34, -2.22044605e-16,  0.00000000e+00,
         0.00000000e+00],
       [-1.61778115e-32,  1.23259516e-31,  1.54074396e-33,
        -1.16573418e-15,  4.93038066e-32,  0.00000000e+00,
         3.69778549e-32, -6.16297582e-33,  1.94289029e-16,
        -6.16297582e-33],
       [-2.66171633e-17,  9.71445147e-17, -7.70371978e-34,
         4.93038066e-32,  3.33066907e-16, -5.55111512e-17,
         1.05471187e-15,  3.08148791e-33,  2.46519033e-32,
         0.

In [78]:
np.allclose(F,h1+J-K/2/4+vxc0 )

True